In [27]:
from pymongo import MongoClient
from IPython.display import JSON
import pandas as pd
from unidecode import unidecode
from joblib import Parallel, delayed

In [16]:
redcol=pd.read_csv("Formatos_Entidades Datos Colav - redcol.csv",header=1)
redcol_ids=[]
for i in redcol["Valor Autorizado REDCOL"].values:
    redcol_ids.append(i.split("/")[-1].lower())


In [20]:
coar=pd.read_csv("Formatos_Entidades Datos Colav - coar.csv",header=0)
coar_ids=[]
for i in coar["@id"].values:
    coar_ids.append(i.lower())

In [24]:
eu_repo=pd.read_csv("Formatos_Entidades Datos Colav - info-eu-repo.csv",header=0)
eu_repo_ids=[]
for i in eu_repo["Term"].values:
    eu_repo_ids.append(i.lower())

In [2]:
def str_normilize(word):
    """
    Normalize a string to lowercase and remove accents.

    Parameters
    ----------
    word : str
        string to be normalized.

    Returns
    -------
    str
        normalized string.
    """
    return unidecode(word).lower().strip().replace(".", "")

In [3]:

def is_thesis(work_type):
    """
    Check if the work type is a thesis.

    Parameters
    ----------
    work_type : str
        work type.

    Returns
    -------
    bool
        True if the work type is a thesis, False otherwise.
    """
    filters=["tesis","thesis","trabajo de grado","monografia","disertacion","monografia","monograph","bachelor"]
    for t in filters:
        if t in str_normilize(work_type.lower()):
            return True
    return False

In [4]:
c=MongoClient()
db=c["oxomoc_colombia"]

In [25]:
cols = sorted([i for i in db.list_collection_names() if "records" in i and "dspace" in i])
cols

['dspace_areandina_records',
 'dspace_artesaniasdecolombia_records',
 'dspace_car_records',
 'dspace_ccb_records',
 'dspace_cecar_records',
 'dspace_ceipa_records',
 'dspace_ces_records',
 'dspace_cesa_records',
 'dspace_corhuila_records',
 'dspace_cuc_records',
 'dspace_cun_records',
 'dspace_ecci_records',
 'dspace_ecr_records',
 'dspace_educacionbogota_records',
 'dspace_eia_records',
 'dspace_elpoli_records',
 'dspace_endeporte_records',
 'dspace_escuelaing_records',
 'dspace_esdegrepositorio_records',
 'dspace_fucsalud_records',
 'dspace_fumc_records',
 'dspace_ibero_records',
 'dspace_icesi_records',
 'dspace_itc_records',
 'dspace_itfip_records',
 'dspace_itm_records',
 'dspace_iue_records',
 'dspace_javeriana_records',
 'dspace_juanncorpas_records',
 'dspace_konradlorenz_records',
 'dspace_lasalle_records',
 'dspace_libertadores_records',
 'dspace_minciencias_records',
 'dspace_pedagogica_records',
 'dspace_poligran_records',
 'dspace_sena_records',
 'dspace_tdea_records',
 'ds

In [51]:
def parse_dspace(
    reg: dict) -> dict:
    found=False
    types=[]
    if "dim:field" in reg["OAI-PMH"]["GetRecord"]["record"]["metadata"]["dim:dim"].keys(): 
        for field in reg["OAI-PMH"]["GetRecord"]["record"]["metadata"]["dim:dim"][
            "dim:field"
        ]:
            if not isinstance(field,dict):
                continue
            # Type
            if field["@element"] == "type":
                if "#text" in field:
                    types.append(field["#text"])
                    t=str_normilize(field["#text"])
                    if "redcol" in t:
                        value = t.split("/")[-1]
                        if value in redcol_ids:
                            found=True
                            
                    if "coar" in t:
                        value = t.split("/")[-1]
                        if value in coar_ids:
                            found=True
                            
                    if "eu-repo" in t or "semantics" in t:
                        value = t.split("/")[-1]
                        if value in eu_repo_ids:
                            found=True
        # si no fue encontrado un tipo de calidad coar, redcol o eu-repo
        # los retorno para ser analizados
        if not found:
            return types


In [52]:
cols

['dspace_areandina_records',
 'dspace_artesaniasdecolombia_records',
 'dspace_car_records',
 'dspace_ccb_records',
 'dspace_cecar_records',
 'dspace_ceipa_records',
 'dspace_ces_records',
 'dspace_cesa_records',
 'dspace_corhuila_records',
 'dspace_cuc_records',
 'dspace_cun_records',
 'dspace_ecci_records',
 'dspace_ecr_records',
 'dspace_educacionbogota_records',
 'dspace_eia_records',
 'dspace_elpoli_records',
 'dspace_endeporte_records',
 'dspace_escuelaing_records',
 'dspace_esdegrepositorio_records',
 'dspace_fucsalud_records',
 'dspace_fumc_records',
 'dspace_ibero_records',
 'dspace_icesi_records',
 'dspace_itc_records',
 'dspace_itfip_records',
 'dspace_itm_records',
 'dspace_iue_records',
 'dspace_javeriana_records',
 'dspace_juanncorpas_records',
 'dspace_konradlorenz_records',
 'dspace_lasalle_records',
 'dspace_libertadores_records',
 'dspace_minciencias_records',
 'dspace_pedagogica_records',
 'dspace_poligran_records',
 'dspace_sena_records',
 'dspace_tdea_records',
 'ds

In [71]:
all_types=[]
#for col_name in ['dspace_univalle_records']:#cols:
for col_name in cols:
    col=db[col_name]
    print(f"processing {col_name}")
    types = Parallel(n_jobs=20,verbose=0,backend="threading")(delayed(parse_dspace)(work) for work in col.find({}))
    types=[i for i in types if i]
    for t in types:
        all_types.extend(t)    


processing dspace_areandina_records
processing dspace_artesaniasdecolombia_records
processing dspace_car_records
processing dspace_ccb_records
processing dspace_cecar_records
processing dspace_ceipa_records
processing dspace_ces_records
processing dspace_cesa_records
processing dspace_corhuila_records
processing dspace_cuc_records
processing dspace_cun_records
processing dspace_ecci_records
processing dspace_ecr_records
processing dspace_educacionbogota_records
processing dspace_eia_records
processing dspace_elpoli_records
processing dspace_endeporte_records
processing dspace_escuelaing_records
processing dspace_esdegrepositorio_records
processing dspace_fucsalud_records
processing dspace_fumc_records
processing dspace_ibero_records
processing dspace_icesi_records
processing dspace_itc_records
processing dspace_itfip_records
processing dspace_itm_records
processing dspace_iue_records
processing dspace_javeriana_records
processing dspace_juanncorpas_records
processing dspace_konradloren

In [72]:
all_types=[i for i in all_types if i]
print(len(all_types))
all_types=list(set(all_types))
len(all_types)

424691


1555

In [73]:
all_types

['Juego',
 'Etnográfico, hisotriográfico heurístico. enfoque cualitativo.',
 'Capitulo - Parte de un Libro',
 'Plan comercial',
 'http://purl.org/coar/resource_type/c_2f193',
 'PT-005',
 'PT-121',
 'Estado del arte',
 'http://purl.org/coar/resource_type/c_2f59',
 'PT-167',
 'Generación de Nuevo Conocimiento: Libros publicados',
 'http://purl.org/coar/resource_type/c_2f136',
 'Instalación',
 'Libro',
 'Trabajo de promoción',
 'Tesis de Tecnología',
 'PT-018',
 'Generación de Nuevo Conocimiento: Obra o creación efímera',
 'Esquema de certificación',
 'http://purl.org/coar/resource_type/c_2f84',
 'Escultura',
 'Ensamble escultórico',
 'PublishedVersion',
 'Apropiación Social y Circulación del Conocimiento: Generación de contenidos impresos, radiales, audiovisuales, multimedia, virtuales y Creative Commons',
 'Análisis cuantitativo',
 'PT-029',
 'Artículo Original',
 'PT-050',
 'PT-138',
 'PT-071',
 'draft',
 'Estudio observacional de una cohorte.',
 'Trabajo',
 'DOSS-CR',
 'http://purl.or

In [74]:
def get_doc(tipo):
    for col_name in cols:
        col=db[col_name]
        rec = col.find_one({"OAI-PMH.GetRecord.record.metadata.dim:dim.dim:field.@element": "type","OAI-PMH.GetRecord.record.metadata.dim:dim.dim:field.#text":tipo})
        if rec:
            return rec
    

In [75]:

all_types = [str_normilize(i) for i in all_types if i]
all_types=list(set(all_types))
len(all_types)

1415

In [76]:
rterm=["fu-","pt-","seccion","version","pre-escolar","http","purl","semantics"]
for term in rterm:
    all_types = [i for i in all_types if term not in i ]


In [77]:
len(all_types)

741

In [78]:
filters=["book","libro","articulo","article","paper","documento","document","monografia","monograph","report",
         "review","revision","ficha","manual","trabajo", "apropiacion","material","object","objeto","caso","project","proyecto","audio","grabación"
        "analisis","investigacion","apropiacion","page","pagina","esquema","work","arte","etno","noticia","thesis","tesis","cuento","mapa",
         "podcast","radio","anal","reseña","curso","nota","product","conferenc","text","infor","memo","obra",
        "produccion","pre-","anima","boletin","capitulo","print","carta","cartilla","guia","cuadern","poema","post","plan","acta",
        "anuario","study","estudio","editorial","video","translation","traduccion","software","ensayo","ponencia","narración",
        "resen","infografia","composicion","bachelor","traduc","data","dato","resume","relato","research","consultoria","disertacion",
        "note","entrevista","folleto","patent","mooc","contribucion","congreso","revista","taller","seminario","event","practica",
        "dossier","coloquio","draft","revisao","aplicativo","web",
        "academic",
 'acuerdo',
 'afiche',
 'agenda',
 'art gallery',
 'articuos evaluados por pares',
 'artigo',
 'artigo de pesquisa',
 'artigo de reflexao',
 'aticulo',
 'bibliography',
 'biografico narrativo',
 'borradores de administracion',
 'busto',
 'cartillero',
 'catalogo',
 'censo',
 'ceramica',
 'cine expandido',
 'coleccion',
 'collection',
 'colofon',
 'colophon',
 'comentario invitado',
 'comites, arbritros, et',
 'comparativo',
 'comunicaciones breves',
 'conjunto escultorico',
 'conjunto escutorico',
 'conservacion',
 'contributiontoperiodical',
 'creacion de redes',
 'creacion o interpretacion',
 'creacion o onterpretacion',
 'debate',
 'decreto',
 'desarrollo tecnologico e innovacion',
 'diagnostico',
 'diagnostico bioquimico por tincion de filipin',
 'diagnostico territorial',
 'diagrama',
 'diario',
 'dibujo',
 'docencia - lectura critica',
 'doss-cr',
 'draf',
 'encuentro',
 'encuentro interdisciplinario',
 'encuesta',
 'encuesta corta',
 'encuesta; observacion directa',
 'encuestas',
 'encuestas a turistas en distintas rutas gastronomicas',
 'ensamble escultorico',
 'errata',
 'erratum',
 'escultura',
 'escultura - talla',
 'escultura pedestre',
 'estatutos',
 'fe de erratas',
 'flujograma',
 'foro',
 'fotografia',
 'fotografias',
 'generacion de nuevo conocimiento: otra publicacion divulgativa',
 'glosario',
 'grabacion sonora',
 'grabaciones sonoras',
 'grafico y simulacion',
 'hermeneutico',
 'hermeneutico, cualitativo',
 'hipermedia narrativo',
 'historia de los imperios; historia cultural de la ciencia',
 'historiografico',
 'image',
 'image, 3-d',
 'imagen',
 'imagen fija',
 'imagen fotografia',
 'imagenes',
 'imagenfotografia',
 'instrucciones a los autores',
 'instructions',
 'interactive resource',
 'interactiveresource',
 'interdisciplinario',
 'internship',
 'interview',
 'investigacao qualitativa',
 'investigacion',
 'investigation',
 'investigative section',
 'investigativo',
 'investiugacion accion educativa',
 'invetigacion cualitativa',
 'journal',
 'lectura',
 'lecturas criticas',
 'legal y contenidos',
 'letter',
 'librillo',
 'literatura gris',
 'magazine',
 'manuscript',
 'manuscripts',
 'manuscrito',
 'map',
 'marco logico',
 'marcos logicos',
 'micrositio',
 'modelo a escala',
 'modelos y prototipos',
 'montaje',
 'montaje escenico',
 'multimedia resource',
 'musical composition',
 'narracion transmedia',
 'netart',
 'norma legal',
 'obituario',
 'other',
 'otre',
 'otro',
 'otros',
 'participacion ciudadana',
 'partitura',
 'partituras',
 'pasantia',
 'performance',
 'periodico',
 'photography',
 'pintura',
 'placa',
 'plataforma web',
 'plegable',
 'poa',
 'portal web',
 'programa radial',
 'protocol',
 'protocolo',
 'protocolos',
 'publicacion academica',
 'publicacion seriada',
 'publisher',
 'recording, oral',
 'reda',
 'referencia bibliografica',
 'referencial',
 'reflection',
 'reflexion',
 'reflexion academica',
 'registro de performance',
 'regulaciones, normas, reglamentos o legislaciones',
 'resolucion',
 'resolucion|',
 'sitio web',
 'stillimage',
 'talla',
 'taxonomia',
 'trajo de grado',
 'website',
]
filtered=[]
for p in all_types:
    for f in filters:
        if f in p :
            filtered.append(p)
len(filtered)
#all_types = list(set(all_types)-set(filtered))

993

In [200]:
len(filtered)

1959

In [88]:
filtered=list(set(filtered))
len(filtered)

615

In [101]:
post_filters=["la investigacion","la presen","this article is the product","institucion 10 4aldocument"]
for term in post_filters:
    filtered = [i for i in filtered if term not in i]
len(filtered)

611

In [102]:
filtered

['fotografias',
 'research report',
 'libros de texto - globalizacion e integracion - relaciones internacionales',
 'tesis de pregrado (monografia)',
 'posters',
 'articulo de periodico',
 'seminario',
 'apropiacion social y circulacion del conocimiento: evento cientifico con componente de apropiacion',
 'proyecto aplicado o tesis',
 'investigacion; cualitativo; cuantitativo',
 'formacion de recurso humano para la ctel: proyectos de investigacion y desarrollo',
 'proyecto aplicado',
 'plano',
 'imagenes',
 'curso de profundizacon',
 'informe de avance',
 'apropiacion social y circulacion del conocimiento: boletin divulgativo de resultado de investigacion',
 'formulacion proyecto',
 'analisis de caso',
 'ensamble escultorico',
 'patente',
 'articulo de investigacion',
 'curso de profundizacioon',
 'baschelor thesis',
 'hermeneutico',
 'tesis pregrado',
 'capitulo de caso',
 'thesis',
 'informe practica',
 'creacion de obra artistica',
 'planos tecnicos',
 'documento historico',
 'recurs

In [103]:
len(filtered)

611

In [104]:
filtered = list(set(filtered))
len(filtered)

611

In [105]:
#set(all_types)-set(filtered)

In [106]:
df=pd.DataFrame()
all_types = [i for i in filtered if i]
all_types=sorted(filtered)
df["source"] = ["dspace"]*len(filtered)
df["tipos_dspace"]=filtered
df.to_excel("tipos_dspace_valid.xlsx")

615

In [108]:
#get_doc("PT-002")